In [0]:
#Changing the working directory to the root
%cd ../..

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
import plotly.io as pio
pio.templates.default = "plotly_white"
pio.kaleido.scope.mathjax = None
import pandas as pd
from pathlib import Path
from tqdm.autonotebook import tqdm
from itertools import cycle
%load_ext autoreload
%autoreload 2
np.random.seed()
tqdm.pandas()

In [0]:
os.makedirs("imgs/chapter_2", exist_ok=True)

### Pandas Datetime Operations, Indexing, and Slicing – A refresher

Before we start wrangling our data, let's quickly look at a few pandas concepts which will be useful. And for that let's pick an easy, well formatted stock exchange price dataset from UCI ML Repo.

In [0]:
df = pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/00247/data_akbilgic.xlsx", skiprows=1)

df.head()

#### Converting date columns to pd.Timestamp/DatetimeIndex
First, we need to convert the date columns in `str` to `pd.Timestamp` and we can use [`pd.to_datetime`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) to accomplish that.

In [0]:
pd.to_datetime("13-4-1987").strftime("%d, %B %Y")

Now let's look at a case where the automatic parsing fails. The date is 4th January, 1987.

In [0]:
pd.to_datetime("4-1-1987").strftime("%d, %B %Y")

Now, anyone can make that mistake because we are not telling the computer whether month comes first or day comes first and pandas assumes month coming first. Let's rectify that.

In [0]:
pd.to_datetime("4-1-1987", dayfirst=True).strftime("%d, %B %Y")

Another case where the automatic date parsing fails is when the date string is in a non-standard form. There we can give a strftime formatted string to help pandas parse the dates correctly

In [0]:
pd.to_datetime("4|1|1987", format="%d|%m|%Y").strftime("%d, %B %Y")

**Best Practice** - Wherever possible, use the `format` or at least `dayfirst` and `yearfirst` parameters to make sure pandas is understanding the dates correctly. `format` is a standard strftime formatted string, for e.g., "%Y-%m-%d". For full table of codes and their meanings refer [here](https://strftime.org/).

In [0]:
df['date'] = pd.to_datetime(df['date'], yearfirst=True)
df['date'].dtype

In [0]:
df.date.min(),df.date.max()

#### dt accessor and datetime properties

In [0]:
print(f"""
Date: {df.date.iloc[0]}
Day of year: {df.date.dt.day_of_year.iloc[0]}
Day of week: {df.date.dt.dayofweek.iloc[0]}
Week of Year: {df.date.dt.weekofyear.iloc[0]}
Month: {df.date.dt.month.iloc[0]}
Month Name: {df.date.dt.month_name().iloc[0]}
Quarter: {df.date.dt.quarter.iloc[0]}
Year: {df.date.dt.year.iloc[0]}
ISO Week: {df.date.dt.isocalendar().week.iloc[0]}
""")

#### Slicing and Indexing

If you set the date column as the index of the dataframe, all kinds of helpful slicing and indexing is possible.

In [0]:
df.set_index("date", inplace=True)

# Select all data after 2010-01-04(including)
df["2010-01-04":]
# Select all data between 2010-01-04 and 2010-02-06(not including)
df["2010-01-04": "2010-02-06"]
# Select data 2010 and before
df[: "2010"]
# Select data between 2010-01 and 2010-06(both including)
df["2010-01": "2010-06"]

#### Sequences and Offsets of Dates

In addition to the semantic information and intelligent indexing and slicing, pandas also has tools to create date sequences and even manipulate date sequences.

In [0]:
# Specifying start and end dates with frequency
pd.date_range(start="2018-01-20", end="2018-01-23", freq="D").astype(str).tolist()

In [0]:
# Specifying start and number of periods to generate in the given frequency
pd.date_range(start="2018-01-20", periods=4, freq="D").astype(str).tolist()

In [0]:
# Generating a date sequence with every 2 days
pd.date_range(start="2018-01-20", periods=4, freq="2D").astype(str).tolist()

In [0]:
# Generating a date sequence every month. By default it starts with Month end
pd.date_range(start="2018-01-20", periods=4, freq="M").astype(str).tolist()

In [0]:
# Generating a date sequence every month, but month start
pd.date_range(start="2018-01-20", periods=4, freq="MS").astype(str).tolist()

We can also add or subtract days/months/etc. to dates using TimeDelta

In [0]:
# Add four days to the date range
(pd.date_range(start="2018-01-20", end="2018-01-23", freq="D") + pd.Timedelta(4, unit="D")).astype(str).tolist()

In [0]:
# Add four weeks to the date range
(pd.date_range(start="2018-01-20", end="2018-01-23", freq="D") + pd.Timedelta(4, unit="W")).astype(str).tolist()

#### Resampling, Shifting, Rolling Window, and Expanding Window Operations

**Resampling**   
Resampling is a powerful operation which let's you aggregate and disaggregate along the temporal dimension.

In [0]:
# Resampling at Month Start and taking the mean
m_df = df[['ISE']].resample("MS").mean()
display(m_df.head())
# Resampling at Week(starting Monday) and taking the mean
w_df = df[['ISE']].resample("W-MON").mean()
display(w_df.head())

In [0]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df.ISE,
    opacity=0.4,
    name="ISE",
    line={"dash": "dot"}
))

fig.add_trace(go.Scatter(
    x=m_df.index,
    y=m_df['ISE'],
    opacity=1.0,
    name="ISE Monthly",
#     line={"dash": "dashdot"}
))
fig.add_trace(go.Scatter(
    x=w_df.index,
    y=w_df['ISE'],
    opacity=1,
    name="ISE Weekly",
    line={"dash": "dash"}
))
fig.show()
fig.write_image("imgs/chapter_2/resampling.png", width=1600, height=600)

**Shift**   
Shifting a series along the temporal dimension is another valuable operation

In [0]:
# fig.update_layout(
#     yaxis=dict(tickfont=dict(size=15)),
#     xaxis=dict(tickfont=dict(size=15)),
#     legend=dict(font=dict(size=15)),
# )
# fig.show()

In [0]:
plot_df = df.loc[:, ['ISE']]
# Shift forward by one day
plot_df['ISE_shift_1'] = df[["ISE"]].shift(1)
# Shift backward by one day
plot_df['ISE_shift_-1'] = df[["ISE"]].shift(-1)

plot_df = plot_df.dropna().iloc[20:40].reset_index()
plot_df = pd.melt(plot_df, id_vars="date")
fig = px.line(plot_df, x="date", y="value", color="variable", line_dash="variable", template="plotly_white")
fig.write_image("imgs/chapter_2/shift.png", width=1600, height=600)
# fig.update_layout(
#     yaxis=dict(tickfont=dict(size=15)),
#     xaxis=dict(tickfont=dict(size=15)),
#     legend=dict(font=dict(size=15)),
# )
fig.show()

**Rolling and Expanding Window**   

There are strong use cases to calculate some statistics on a rolling or expanding window.

In [0]:
plot_df = df.loc[:, ['ISE']]
# Moving Average of 5 days
plot_df["ISE_rolling_5_mean"] = df[['ISE']].rolling(window=5).mean()

plot_df = plot_df.dropna().iloc[20:50].reset_index()
plot_df = pd.melt(plot_df, id_vars="date")
fig = px.line(plot_df, x="date", y="value", line_dash="variable", template="plotly_white")
# fig.update_layout(
#     yaxis=dict(tickfont=dict(size=15)),
#     xaxis=dict(tickfont=dict(size=15)),
#     legend=dict(font=dict(size=15)),
# )
fig.write_image("imgs/chapter_2/rolling_1.png", width=1600, height=600)
fig.show()

In [0]:
plot_df = df.loc[:, ['ISE']]
# Moving Average of 30 days
plot_df["ISE_rolling_30_mean"] = df[['ISE']].rolling(window=120).mean()
# Expanding Average
plot_df["ISE_expanding_mean"] = df[['ISE']].expanding().mean()
plot_df.drop(columns='ISE', inplace=True)
plot_df = plot_df.dropna().reset_index()
plot_df = pd.melt(plot_df, id_vars="date")
fig = px.line(plot_df, x="date", y="value", line_dash="variable", template="plotly_white")
# fig.update_layout(
#     yaxis=dict(tickfont=dict(size=15)),
#     xaxis=dict(tickfont=dict(size=15)),
#     legend=dict(font=dict(size=15)),
# )
fig.write_image("imgs/chapter_2/rolling_2.png", width=1600, height=600)
fig.show()

In [0]:
del plot_df, df, w_df

### Missing Value Treatment

In [0]:
def format_plot(fig, legends, font_size=15, title_font_size=20):
    names = cycle(legends)
    fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": title_font_size},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text="Value",
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text="Day",
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig

In [0]:
df = pd.read_csv("https://www.data.act.gov.au/resource/94a5-zqnn.csv", sep=",")
#Selecting one location and pm2.5
df = df.loc[df.name=="Monash", ['datetime', 'pm2_5_1_hr']]
df.datetime = pd.to_datetime(df.datetime)
df.sort_values("datetime", inplace=True)
df.set_index("datetime", inplace=True)
# introducing a missing value by nulling out the values
df.loc["2022-07-07 7:00":"2022-07-07 09:00", "pm2_5_1_hr"] = np.nan
df.head()

**We have introduced a few missing values between "2022-07-07 00:00" and "2021-07-07 12:00" in the PM2.5 column**

In [0]:
df = df.loc["2022-07-07":"2022-07-07 12:00", "pm2_5_1_hr"].to_frame()
fig = px.line(df, x=df.index, y="pm2_5_1_hr", title="Missing Values in PM2.5")
fig = format_plot(fig, ["Original"])
fig.write_image("imgs/chapter_2/missing_values.png")
fig.show()

## Last Observation Carried Forward (Forward Fill), Next Observation Carried Backward (Backward Fill) and Mean Value Fill

In [0]:
_df = df.copy()
#Forward Fill
_df["ffill"] = _df['pm2_5_1_hr'].ffill()
# Backward Fill
_df["bfill"] = _df['pm2_5_1_hr'].bfill()
# Mean Value Fil
_df["mean_value"] = _df['pm2_5_1_hr'].fillna(df['pm2_5_1_hr'].mean())

In [0]:
#Plotting
plot_df = pd.melt(_df.reset_index(), id_vars="datetime", var_name="series")
fig = px.line(plot_df, x="datetime", y=["value"], line_dash="series", title="Forward, Backward, and Mean Value Fill")
fig = format_plot(fig, ['Original', 'Forward Fill', 'Backward Fill', "Mean Value Fill"])
fig.write_image("imgs/chapter_2/ffill_bfill_mean_missing_values.png")
fig.show()

## Interpolation

In [0]:
_df = df.copy()
_df["linear_interpolation"] = _df['pm2_5_1_hr'].interpolate(method="linear")
_df["nearest_interpolation"] = _df['pm2_5_1_hr'].interpolate(method="nearest")

In [0]:
#Plotting
plot_df = pd.melt(_df.reset_index(), id_vars="datetime", var_name="series")
fig = px.line(plot_df, x="datetime", y=["value"], line_dash="series", title="Linear and Nearest Interpolation")
fig = format_plot(fig, ['Original', "Linear Interpolation", "Nearest Interpolation"])
fig.write_image("imgs/chapter_2/linear_nearest_missing_values.png")
fig.show()

In [0]:
_df = df.copy()
_df["spline_interpolation"] = _df['pm2_5_1_hr'].interpolate(method="spline", order=2)
_df["polynomial_interpolation"] = _df['pm2_5_1_hr'].interpolate(method="polynomial", order=5)

In [0]:
#Plotting
plot_df = pd.melt(_df.reset_index(), id_vars="datetime", var_name="series")
fig = px.line(plot_df, x="datetime", y=["value"], line_dash="series", title="Spline and Polynomial Interpolation")
fig = format_plot(fig, ['Original', "2nd Order Spline Interpolation", "5th Order Polynomial Interpolation"])
fig.write_image("imgs/chapter_2/spline_polynomial_missing_values.png")
fig.show()